In [4]:
from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    return preprocess_text(ref, 
                           # replace accenteds with unaccented versions
                           no_accents=True,
                           # lowercase
                           lowercase=True, 
                           # replace punctuation with empty string
                           no_punct=True)

In [2]:
SUBMISSION_FILEPATHS = [
    "../data/webnlg2017/submissions/melbourne/final_result.txt",
    "../data/webnlg2017/submissions/tilburg/smt_test.out.ordered",
    "../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt",
    "../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt",
    "../data/webnlg2017/submissions/tilburg/template_test.out.ordered",
    "../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered",
    "../data/webnlg2017/submissions/baseline_sorted.txt",
    "../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt",
    "../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt"
]

# BLEU

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

In [17]:
!ls ../paper/webnlg2017/

The WebNLG Challenge - Generating Text from DBPedia Data.pdf
The WebNLG Challenge -  Generating Text from RDF Data.pdf
WebNLG Challenge - Human Evaluation.pdf


## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/bleu_result.jpg' />

## Melbourne

In [1]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt

BLEU = 45.13, 79.9/55.4/38.8/27.4 (BP=0.969, ratio=0.969, hyp_len=40642, ref_len=41921)


## Tilb-SMT

In [4]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-2_all-cat.txt

BLEU = 44.28, 74.4/53.0/37.5/26.8 (BP=0.992, ratio=0.992, hyp_len=42193, ref_len=42521)


## PKUWriter

In [5]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt

BLEU = 39.88, 73.9/49.9/35.6/26.0 (BP=0.927, ratio=0.930, hyp_len=38248, ref_len=41143)


## UPF-FORGe

In [6]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt

BLEU = 38.65, 75.5/48.7/31.1/19.5 (BP=1.000, ratio=1.064, hyp_len=47234, ref_len=44405)


## Tilb-Pipeline

In [8]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt

BLEU = 35.29, 81.8/56.6/38.8/26.5 (BP=0.755, ratio=0.781, hyp_len=30335, ref_len=38854)


## Tilb-NMT

In [9]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt

BLEU = 34.60, 68.8/42.7/27.5/17.7 (BP=1.000, ratio=1.054, hyp_len=45605, ref_len=43278)


## Baseline

In [10]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt

BLEU = 33.24, 65.4/42.3/31.8/24.8 (BP=0.865, ratio=0.873, hyp_len=35312, ref_len=40435)


## Adapt

In [11]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt

BLEU = 31.06, 54.9/35.7/25.3/18.7 (BP=1.000, ratio=1.337, hyp_len=59727, ref_len=44663)


## UIT-VNU

In [12]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt

BLEU = 7.07, 67.5/54.4/39.5/29.2 (BP=0.156, ratio=0.350, hyp_len=11219, ref_len=32072)
